In [4]:
import tensorflow as tf 

A dataset comprises elements that each have the same structure. An element contains one or more `tf.Tensor` objects, called **components**.

In [2]:

dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 5, 10]))
print(dataset1.output_types)
print(dataset1.output_shapes)
print(dataset1.output_classes)

<dtype: 'float32'>
(5, 10)
<class 'tensorflow.python.framework.ops.Tensor'>


In [5]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]),
    tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)))

print(dataset2.output_shapes)
print(dataset2.output_types)

(TensorShape([]), TensorShape([Dimension(100)]))
(tf.float32, tf.int32)


It's often convinient to give names to each **component** of an element, for example if they represent different features of a training example.

In addtion to tupples, you can use `collections.namedtuple` or a dictionary mapping strings to tensors to represent a single element of a `Dataset`. 

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "a": tf.random_uniform([4]),
        "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)
    }
)

print(dataset.output_shapes)
print(dataset.output_types)

{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}
{'a': tf.float32, 'b': tf.int32}


## Creating an iterator

Once you have built a `Dataset` to represent your input data, the next step is to 
create an `Iterator` to access elements from that dataset.

The `tf.data` API currently supports the following iterations:
* **one-shot**


In [10]:
dataset = tf.data.Dataset.range(10)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(10):
        value = sess.run(next_element)
        print(value)

0
1
2
3
4
5
6
7
8
9


An **initializable** iterator requires you to run an explicit `iterator.initializer` operation before using it. In exchange for this inconvenience, it enables you to parameterize the definition of the dataset using one or more `tf.placeholder()` tensors that can be fed when you initialize the iterator.

In [13]:
max_value = tf.placeholder(tf.int64, shape=[])

dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Initialize an iterator over a dataset with 10 elements.
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={
        max_value: 10
    })
    for i in range(10):
        value = sess.run(next_element)
        print(value)
        
# Initialize the same iterator over a dataset with 5 elements.
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={
        max_value: 5
    })
    for i in range(5):
        value = sess.run(next_element)
        print(value)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
